In [3]:
from client import Client
from server import Server
from sklearn.metrics import f1_score, accuracy_score
from sklearn.datasets import load_iris
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np

In [4]:
test = [1,2,3,4,5]
test[:-1]

[1, 2, 3, 4]

In [5]:
alice = Client()
server = Server()

In [6]:
pseudo_x_train = [[1,2,3,4,5], [6,7,8,9,10],[90,91,92,93,94], [100,101,102,103,104]]
pseudo_y_train = [0,0,1,1]
pseudo_test_row = [10,11,12,13,14]

In [7]:
server.set_features_labels(pseudo_x_train, pseudo_y_train)

In [8]:
print(server.features)
print(server.labels)

[[1, 2, 3, 4, 5], [6, 7, 8, 9, 10], [90, 91, 92, 93, 94], [100, 101, 102, 103, 104]]
[0, 0, 1, 1]


In [9]:
alice.predict_classification(pseudo_test_row, 3, server, 4, 4)

0

In [10]:
# test multiple points

def test_multiple_points(X_train, X_test, y_train, y_test, num_neighbors, client: Client, server: Server, rand_range):
	server.set_features_labels(X_train, y_train)
	y_pred = []
	for test_row in X_test:
		prediction = client.predict_classification(test_row, num_neighbors, server, rand_range)
		y_pred.append(prediction)

	f1_binary = f1_score(y_test, y_pred, average="binary")
	accuracy = accuracy_score(y_test, y_pred)

	# print("ypred:", y_pred)
	# print("ytest:", y_test)

	return f1_binary, accuracy

In [11]:
# get iris dataset
X_class,y_class = load_iris().data, load_iris().target

# we just want binary classification
X_class = X_class[:100]
y_class = y_class[:100]

X_class,y_class = shuffle(X_class, y_class)

x_df = pd.DataFrame(X_class)
y_df = pd.DataFrame(y_class)

In [15]:
#testing

def test(rand_range):

    numFolds = 10
    num_neighbors = 9
    stratifiedKFold = StratifiedKFold(
        n_splits=numFolds, shuffle=True, random_state=86
    )

    count = 1
    avgF1 = 0
    avgAcc = 0

    X = X_class
    y = y_class

    server = Server()
    alice = Client()
    rand_seed = 4
    rand_range = 100

    for train_index, test_index in stratifiedKFold.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        f1, acc = test_multiple_points(X_train, X_test, y_train, y_test, num_neighbors, alice, server, rand_range)
        avgF1 += f1
        avgAcc += acc

        #print(f"\tFold {count}:: Accuracy: {acc}, F1 score: {f1}")

        count += 1

    avgF1 = avgF1/numFolds
    avgAcc = avgAcc/numFolds

    return avgAcc, avgF1



In [18]:
# test with multiple lagrange max x-points

x_max_list = [100,1000,10000,100000,1000000]

print("Testing Iris Dataset - SMPC")

for x_max in x_max_list:
    avgAcc, avgF1 = test(x_max)
    print(f"Average accuracy: {avgAcc}, Average F1 score: {avgF1}, Lagrange max x-coord: {x_max}")

Testing Iris Dataset - SMPC
Average accuracy: 1.0, Average F1 score: 1.0, Lagrange max x-coord: 100
Average accuracy: 1.0, Average F1 score: 1.0, Lagrange max x-coord: 1000
Average accuracy: 1.0, Average F1 score: 1.0, Lagrange max x-coord: 10000
Average accuracy: 1.0, Average F1 score: 1.0, Lagrange max x-coord: 100000
Average accuracy: 1.0, Average F1 score: 1.0, Lagrange max x-coord: 1000000
